In [2]:
from pathlib import Path
import urllib.request

import gensim
import numpy as np
import pandas as pd
from keras.layers import (Dense, Input, 
    GlobalMaxPooling1D, Conv1D, Embedding)
from keras.models import Model
from keras.optimizers import RMSprop
from keras.preprocessing.sequence import (
    pad_sequences)
from keras.preprocessing.text import Tokenizer

/home/wva/ccsbook/env/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
2021-07-06 22:45:34.922523: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-07-06 22:45:34.922580: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
url="http://cssbook.net/d/sentiment_headlines.csv"
h = pd.read_csv(url)
h.head()

,id,value,lemmata
0,10007,0,Rabobank voorspellen flink stijging hypotheekr...
1,10027,0,D66 willen reserve provincie aanspreken voor g...
2,10037,1,UWV dit jaar veel baan
3,10059,1,proost op geslaagd beursgang bols
4,10099,0,helft werknemer gaan na 65ste met pensioen


In [10]:
# Prepare embeddings layer
embeddings = gensim.models.Word2Vec.load("w2v_320d")
"bla" in embeddings.wv




AttributeError: 'KeyedVectors' object has no attribute 'key_to_index'

In [7]:
# Tokenize texts
tokenizer=Tokenizer(num_words=9999)
tokenizer.fit_on_texts(h.lemmata)
word_index=tokenizer.word_index
sequences=tokenizer.texts_to_sequences(h.lemmata)
tokens=pad_sequences(sequences, maxlen=1000)

# Prepare embeddings layer
embeddings_file = Path("w2v_320d")
if (not embeddings_file.exists()):
  print("Downloading embeddings (patience...)")
  url = "http://i.amcat.nl/w2v_320d"
  urllib.request.urlretrieve(url, embeddings_file)
print("Loading embeddings (patience...)")
embeddings = gensim.models.Word2Vec.load(
  str(embeddings_file))
emb_matrix = np.zeros(
  (len(tokenizer.word_index) + 1, 
   embeddings.vector_size))
for word, i in tokenizer.word_index.items():
  if word in embeddings.wv:
      emb_matrix[i] = embeddings.wv[word]
embedding_layer = Embedding(emb_matrix.shape[0],
                            emb_matrix.shape[1],
                            weights=[emb_matrix],
                            input_length=tokens.shape[1],
                            trainable=True)

print("Build RNN model")
sequence_input = Input(shape=(tokens.shape[1],), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
model = Conv1D(filters=128, kernel_size=3, activation='relu')(embedded_sequences)
model = GlobalMaxPooling1D()(model)
model = Dense(64, activation='relu')(model)
preds = Dense(1, activation='tanh')(model)
model = Model(sequence_input, preds)
model.summary()

Loading embeddings (patience...)


AttributeError: 'KeyedVectors' object has no attribute 'key_to_index'

In [4]:
# Split data into train and test
train_data = tokens[:4000]
test_data = tokens[4000:]
train_labels = h.value[:4000]
test_labels = h.value[4000:]

print("Train model")
optimizer = RMSprop(lr=.004)
model.compile(loss='mean_absolute_error', optimizer=optimizer)
labels = np.asarray([[x] for x in train_labels])
model.fit(train_data, labels, epochs=5, batch_size=128)

print("Validate against test data")
output = model.predict(test_data)
# Bin output into -1, 0, 1
predictions = [x[0] > .33 and 1 or (x[0] < -.33 and -1 or 0) 
               for x in output]

acc = (sum([x == y for (x, y) in zip(predictions, test_labels)]) 
       / len(predictions))
print(f"Accuracy: {acc}")

Train model
Epoch 1/5
32/32 [==============================] - 27s 832ms/step - loss: 0.7204
Epoch 2/5
32/32 [==============================] - 25s 782ms/step - loss: 0.4850
Epoch 3/5
32/32 [==============================] - 27s 842ms/step - loss: 0.3335
Epoch 4/5
32/32 [==============================] - 25s 786ms/step - loss: 0.2358
Epoch 5/5
32/32 [==============================] - 26s 806ms/step - loss: 0.1632
Validate against test data
Accuracy: 0.5831180017226529
